# Main Notebook Of the Thesis

This notebook contains the general results on the thesis exploiting the simulator. It is divided in three sections:
1) Campaign Simulation
2) Campaign fitting through HMM
3) Budget Optimization

In [2]:
# Built-in libraries
import matplotlib.pyplot as plt
import numpy as np
import random
import importlib
from joblib import delayed, Parallel

# Constants
import config.execution_parameters as consts
# Project libraries
import src.simulator_package.simulator_functions
from src.simulator_package.simulator_functions import *
import src.hmm_package.generate_hmm
from src.hmm_package.generate_hmm import *
import src.plot_and_print_info.plots_and_print_info
from src.plot_and_print_info.plots_and_print_info import *
import src.optimization_package.optimizer
from src.optimization_package.optimizer import OptimizerPGPE

In [3]:
#Reload libraries
importlib.reload(consts)
from config.execution_parameters import *

importlib.reload(src.hmm_package.generate_hmm)
from src.hmm_package.generate_hmm import *

importlib.reload(src.simulator_package.simulator_functions)
from src.simulator_package.simulator_functions import *

importlib.reload(src.plot_and_print_info.plots_and_print_info)
from src.plot_and_print_info.plots_and_print_info import *

importlib.reload(src.optimization_package.optimizer)
from src.optimization_package.optimizer import OptimizerPGPE

In [4]:
# Store constants
cm = {} # Constants Manager
for k, v in consts.__dict__.items():
    if not k.startswith('__'):
        cm[k] = v

## 1) Advertising Campaign Simulation

In [99]:
# Generate the Campaign with all subcampaigns
advertising_campaign = SimulationClass(cm, cm['ANALYSIS_MODE'])

In [100]:
# Print the information
advertising_campaign.campaigns_list

{0: {'name': 'camp_awn_1',
  'target': 'awareness',
  'who_can_affect_me': [],
  'prob_exposition': [0.5803539961055425],
  'influence': [0.5803539961055425]},
 1: {'name': 'camp_awn_2',
  'target': 'awareness',
  'who_can_affect_me': [],
  'prob_exposition': [0.6013888304042038],
  'influence': [0.6013888304042038]},
 2: {'name': 'camp_traff_1',
  'target': 'traffic',
  'who_can_affect_me': range(0, 2),
  'prob_exposition': [0.41313531244263363],
  'influence': [0.41313531244263363]},
 3: {'name': 'camp_traff_2',
  'target': 'traffic',
  'who_can_affect_me': range(0, 2),
  'prob_exposition': [0.43448018108712405],
  'influence': [0.43448018108712405]},
 4: {'name': 'camp_cnv_1',
  'target': 'conversion',
  'who_can_affect_me': range(0, 4),
  'prob_exposition': [0.1963428314617387],
  'influence': [0.1963428314617387]},
 5: {'name': 'camp_cnv_2',
  'target': 'conversion',
  'who_can_affect_me': range(0, 4),
  'prob_exposition': [0.3090200387981509],
  'influence': [0.3090200387981509]}

In [101]:
# Generate the simulation instantiating the SimulationClass and calling its method 'simulate'
advertising_campaign.simulate(cm)

# Transform the attributes of the campaign into variables that can be used in the HMM fitting Model
adstock = compute_adstock(cm, observation=advertising_campaign.results["user_expositions"])
output = np.append( np.zeros([adstock.shape[0], 1]), advertising_campaign.results["user_outcome"], axis=1 )

In [102]:
# Compute the conversion rate which will be the benchmark for the optimization
count_conversions(output)

Percentage of conversion is: 17.529165420281185%.


17.529165420281185

## 2) Fit the Hidden Markov Model

In [146]:
# Initialize Variables
MU = [ -3., -1.1, -6. ]
BETA = [1.7, -0.2, 0.25,
        1.7, -0.2, 0.25,
        1.1, -0.3, 0.7,
        1.1, -0.3, 0.7,
        0.6, -0.4, 1.1,
        0.6, -0.4, 1.1]
inverse_sigmoid = lambda x: np.log(x/(1-x))
# Now try to fit the model sampled from the real parameters.
initializer={
    'MU': tf.keras.initializers.Constant(MU),
    'BETA': tf.keras.initializers.Constant(BETA),
    'INIT_PROB': tf.keras.initializers.Constant(inverse_sigmoid(0.9)), #
    'CLICK_PROB': tf.keras.initializers.Constant(inverse_sigmoid(0.9))},
model = build_hmm_to_fit_beta(cm, initializer=initializer )

compiler = CompilerInfoBeta(cm, cm['LR_EXPONENTIAL_DECAY'])
model.compile(
    loss = compiler.loss,
    optimizer = compiler.optimizer,
    run_eagerly = True
)
# Fit the model on the data
history = fit_model(cm, model, adstock, output)

Mu: [-3.0, -1.1, -6.0] Beta: [1.7, -0.2, 0.25, 1.7, -0.2, 0.25, 1.1, -0.3, 0.7, 1.1, -0.3, 0.7, 0.6, -0.4, 1.1, 0.6, -0.4, 1.1] Init Prob: [<tf.Tensor: shape=(), dtype=float32, numpy=0.90000004>] Click Prob: [<tf.Tensor: shape=(), dtype=float32, numpy=0.90000004>]
Epoch 1/100
20/20 [==============================] - 9s 447ms/step - loss: 8529.8877
Mu: [-3.0198905, -1.0802078, -6.0198336] Beta: [1.6864698, -0.21999778, 0.23016559, 1.6913843, -0.22000945, 0.23036085, 1.0875915, -0.31479937, 0.68057734, 1.0983078, -0.3026237, 0.68145704, 0.5801775, -0.38048324, 1.0801724, 0.5806841, -0.38111883, 1.0802522] Init Prob: [<tf.Tensor: shape=(), dtype=float32, numpy=0.9017837>] Click Prob: [<tf.Tensor: shape=(), dtype=float32, numpy=0.90175873>]
Epoch 2/100
20/20 [==============================] - 8s 395ms/step - loss: 8369.7598
Mu: [-3.0392666, -1.0610393, -6.0391526] Beta: [1.6805404, -0.23989838, 0.21100806, 1.6959394, -0.24015267, 0.21125701, 1.0784075, -0.33101785, 0.6616343, 1.1010493, -0

In [147]:
# Store parameters in the constants
cm['MU'] = model.weights[0].numpy()
cm['BETA'] = model.weights[1].numpy()
cm['INITIAL_STATE_PROB'] = [1/(1+np.exp(-model.weights[2].numpy()))]
cm['CLICK_PROB'] = [1/(1+np.exp(-model.weights[3].numpy()))]

# 3) Optimize the Campaign Budget

In [69]:
# Optimize the campaign

# initialize with uniform distribution
init_values = np.array([1350, 1350, 1350, 1350, 1350, 1350])
policy = OptimizerPGPE(cm, num_sample=36, hmm_trials=20, N_users_to_simulate=20000, init_values=init_values)

In [43]:
budgets_list = []
convs_list = np.array([])
learning_rate = 1

while True:
    budgets = policy.get_theta()
    returns = Parallel(n_jobs=4)(delayed(policy.act)(budget) for budget in budgets)
    #returns = [policy.act(budget) for budget in budgets]
    print(returns)

    delta_params = np.array(policy.eval_gradient(budgets, returns, use_baseline=True))

    print('Budget distribution: ', policy.get_budget_distribution(), 'Return: ', policy.act(policy.get_budget_distribution()),
          'Variance: ', policy.variance)

    params = policy.get_params()
    policy.set_params(params + learning_rate*delta_params)

    learning_rate = max(0.95*learning_rate, 1e-3)

    if max(policy.variance) < 5e-2:
        break

2022-06-30 11:39:31.575718: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-30 11:39:31.611216: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-30 11:39:31.808775: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

[0.24285, 0.1516975, 0.219195, 0.2702675, 0.28742, 0.0361375, 0.121545, 0.2693, 0.236575, 0.2171225, 0.1718925, 0.256995, 0.17811, 0.26103, 0.2685075, 0.1711725, 0.226085, 0.31331, 0.2219825, 0.2676475, 0.2687075, 0.1750675, 0.27846, 0.2853275, 0.2262075, 0.2242375, 0.2569475, 0.101655, 0.0890075, 0.25337, 0.21057, 0.2516, 0.2590825, 0.207125, 0.24394, 0.2092975]
Budget distribution:  [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667] Return:  0.233345 Variance:  [4.28571429 4.28571429 4.28571429 4.28571429 4.28571429 4.28571429]
[0.1870875, 0.259835, 0.0532225, 0.2977725, 0.109255, 0.21602, 0.2126425, 0.1634125, 0.1578925, 0.2354125, 0.2631025, 0.203005, 0.1975575, 0.1987125, 0.2146375, 0.213305, 0.2535225, 0.223495, 0.2018925, 0.2935525, 0.142495, 0.1545425, 0.2490125, 0.2213575, 0.229165, 0.17098, 0.194145, 0.2178475, 0.092555, 0.20071, 0.2430775, 0.25891, 0.2501875, 0.1971125, 0.2605625, 0.2048525]
Budget distribution:  [0.16857106 0.16932037 0.16280984 0.16583008 

In [45]:
# Assign Variables from the policy found from the PGPE to regenerate the campaign in the simulator
budget_optimized = policy.get_budget_distribution()
complete_budget_distribution = budget_optimized*sum(init_values)
impressions = [round(impression) for impression in complete_budget_distribution]
cm['campaigns']['n_awareness_1'] = impressions[0]
cm['campaigns']['n_awareness_2'] = impressions[1]
cm['campaigns']['n_traffic_1'] = impressions[2]
cm['campaigns']['n_traffic_2'] = impressions[3]
cm['campaigns']['n_conversion_1'] = impressions[4]
cm['campaigns']['n_conversion_1'] = impressions[5]

optimized_campaign = SimulationClass(cm, cm['ANALYSIS_MODE'])
for idx in range(cm['N_camp']):
    optimized_campaign.campaigns_list[idx]['influence'] =  advertising_campaign.campaigns_list[idx]['influence']


In [74]:
# Print the new distribution of the impressions
print(impressions)

[1327, 3090, 196, 136, 117, 3234]

Generate the new campaign with the optimized impressions, and print the conversion rate obtained

In [76]:
# Generate the simulation instantiating the SimulationClass and calling its method 'simulate'
optimized_campaign.simulate(cm)

adstock = compute_adstock(cm, observation=optimized_campaign.results["user_expositions"])
output = np.append( np.zeros([adstock.shape[0], 1]), optimized_campaign.results["user_outcome"], axis=1 )
count_conversions(output)

Percentage of conversion is: 25.9682224428997%.


25.9682224428997